In [21]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import numpy as np

# Veri seti yolu
data_path = 'denemeler/'

# Veri setini yükleme ve ön işleme
train_datagen = ImageDataGenerator(
    rescale=1./255,
    validation_split=0.2
)

train_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    subset='training'
)

validation_generator = train_datagen.flow_from_directory(
    data_path,
    target_size=(250, 250),
    batch_size=32,
    class_mode='categorical',
    subset='validation'
)

# DenseNet121 modelini oluşturma
base_model = tf.keras.applications.DenseNet121(input_shape=(250, 250, 3),
                                               include_top=False,
                                               weights='imagenet')

model = tf.keras.Sequential([
    base_model,
    tf.keras.layers.GlobalAveragePooling2D(),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dropout(0.5),
    tf.keras.layers.Dense(train_generator.num_classes, activation='softmax')
])

# Modeli derleme
model.compile(optimizer='adam',
              loss='categorical_crossentropy',
              metrics=['accuracy'])

Found 10405 images belonging to 2602 classes.
Found 1713 images belonging to 2602 classes.


In [22]:
# Modeli eğitme
history = model.fit(train_generator,
                    steps_per_epoch=train_generator.samples // train_generator.batch_size,
                    epochs=5,
                    validation_data=validation_generator,
                    validation_steps=validation_generator.samples // validation_generator.batch_size)

# Eğitim verileri üzerinde tahmin yapma
y_pred_train = model.predict(train_generator)
y_pred_train = np.argmax(y_pred_train, axis=1)

# Gerçek etiketleri alma
y_true_train = train_generator.classes

# Metrikleri hesapla
train_accuracy = accuracy_score(y_true_train, y_pred_train)
f1_micro = f1_score(y_true_train, y_pred_train, average='micro')
f1_macro = f1_score(y_true_train, y_pred_train, average='macro')
precision = precision_score(y_true_train, y_pred_train, average='weighted', zero_division=0)
recall = recall_score(y_true_train, y_pred_train, average='weighted', zero_division=0)

print(f'Eğitim Doğruluk: {train_accuracy}')
print(f'F1 Mikro: {f1_micro}')
print(f'F1 Makro: {f1_macro}')
print(f'Precision: {precision}')
print(f'Recall: {recall}')

Epoch 1/5
325/325 [==============================] - 1393s 4s/step - loss: 4.2755 - accuracy: 0.5021 - val_loss: 15.6407 - val_accuracy: 0.7659
Epoch 2/5
325/325 [==============================] - 1293s 4s/step - loss: 4.1103 - accuracy: 0.5088 - val_loss: 2.1895 - val_accuracy: 0.7700
Epoch 3/5
325/325 [==============================] - 1330s 4s/step - loss: 4.0855 - accuracy: 0.5113 - val_loss: 1.8628 - val_accuracy: 0.7665
Epoch 4/5
325/325 [==============================] - 1337s 4s/step - loss: 4.0313 - accuracy: 0.5121 - val_loss: 1.6232 - val_accuracy: 0.7818
Epoch 5/5
326/326 [==============================] - 317s 967ms/step
Eğitim Doğruluk: 0.5045651129264777
F1 Mikro: 0.5045651129264777
F1 Makro: 0.00025776796022812217
Precision: 0.25458595318250915
Recall: 0.5045651129264777


In [23]:
fscore = 2 * (precision * recall) / (precision + recall)
print(f'F1 Score: {fscore}')

F1 Score: 0.33841799333938133
